### If you are reading this notebook on the GitHub, please go to [README](./README.md) and follow installation instructions to set everything up locally, it's an interactive notebook and you need a local setup to execute the cells. 

---

CS6601 - Trail Isolation!

---
## Assignment Instructions

### Assignment Description

The goals of this assignment are as follows:

1. Verify student understanding of **foundational adversarial game search algorithms** introduced in lecture. (Minimax)
2. Explore **improved adversarial search algorithms** that build upon the foundational adversarial search algorithms. (AlphaBeta)

Your task is to create an AI that can play and win a game of Trail Isolation. Your AI will be tested against several pre-baked AIs. You will implement your AI in Python 3.9, using our provided code as a starting point. 

In case we haven't got this into your heads enough: **start early!!!** It is entirely possible, and probably likely, that a large part of your next 2 weeks will be devoted to this assignment, but we hope that it is worth it and you will really enjoy it! 

Good luck!

---

### About the Game

**Trail Isolation rules**

- **Pieces & board:** Two players, one queen each, on an \(N x N\) grid (board size per `isolation.py`).
- **Queen movement:** On your turn, move your queen like a chess queen: any number of open squares **horizontally, vertically, or diagonally**, without jumping over blocked or occupied squares.
- **Permanent block:** After you move, the square you moved from becomes **permanently blocked** for the rest of the game.
- **Temporary “trail” block:** All intermediate squares your queen traversed during that move (the path squares between origin and destination) become **temporarily unavailable** for exactly one opponent turn.  
  - These trail squares are unavailable to your opponent during the opponent’s immediate turn (can’t move onto or through them).  
  - After the opponent completes their move, those trail squares **automatically revert to open**.
- **Destination:** Your queen occupies the destination square (as usual).
- **No passing:** If a player has **no legal move**, that player **loses**.
- **Setup:** Starting placement and board initialization are randomized and handled by the provided code.

**Example:**  
If you move from $(r,c)$ to $(r,c+3)$ along the row, then:
- $(r,c)$ becomes **permanently blocked**.  
- $(r,c+1)$ and $(r,c+2)$ are the trail and are **temporarily blocked** for the opponent’s next turn only.  
- $(r,c+3)$ is your queen’s new position.

---

### Table of contents

0. [Grading & Submission Policies](#Grading-&-Submission-Policies)
1. [Additional Helper Files](#Additional-Helper-Files)
2. [About the Game](#About-the-Game)
3. [Assignment contents](#Assignment-contents)
4. [Exporting the notebook](#Exporting-the-notebook)
5. [Coding time!](#Coding-time!)
6. [Section2a checkpoint!](#Section-2a-Checkpoint)
7. [Section2b checkpoint!](#Section-2b-Checkpoint)
8. [Section2c checkpoint!](#Section-2c-Checkpoint)
9. [Bot fight!](#Botfight-(Extra-Credit))

### Grading & Submission Policies

The grade you receive for the assignment will be determined as follows:

| Section | Points    | Condition |
| ------- | --------- | --------- |
| 2a | 5 points  | You write an evaluation function, OpenMoveEval, which returns the number of moves that the AI minus the number of moves opponent can make, and your evaluation function performs correctly on some sample boards we provide. |
| 2a | 30 points | Your AI defeats a random player >= 90% of the time. |
| 2b | 20 points | Your AI defeats an agent with OpenMoveEval function that uses minimax to level 2  >= 80% of the times. |
| 2b | 20 points | Your AI defeats an agent with OpenMoveEval function that uses alphabeta to level 4  >= 70% of the times. |
| 2c | 20 points | Your AI defeats an agent with OpenMoveEval function that uses iterative deepening and alpha-beta pruning >= 70% of the time. |
| 2c | 5 points  | Your AI defeats an agent with Peter's secret evaluation function that uses iterative deepening and alpha-beta pruning and optimizes various aspects of the game player >= 80% of the time  |

As you can see from the table there are three autograded sections, each having the following submission frequency restrictions:
- **Section 2a** - 1 submission per 30 minutes.
- **Section 2b** - 6 submissions per 360 minutes.
- **Section 2c** - 6 submissions per 360 minutes.

### Additional Helper Files
While you'll only have to edit `notebook.ipynb` and submit the exported `submission.py`, there are a number of notable files:

1. `isolation.py`: Includes the `Board` class and a function for printing out a game as text. Do **NOT** change contents of this file. We have the same file on the server's side, so any changes will not be accounted for.
2. `notebook.ipynb`: Where you'll implement the required methods for your agents.
3. `player_submission_tests.py`: Sample tests to validate your agents locally.
4. `test_players.py`: Contains 2 player types for you to test agents locally:
    - `RandomPlayer` - chooses a valid move randomly from among the available valid moves
    - `HumanPlayer` - allows *YOU* to play against the AI in terminal (else use `InteractiveGame` in jupyter)

Additionally, we've included a number of local unit tests to help you test your player and evaluation function as well as to give you an idea of how the classes are used.

## Setup Verification

First, let's make sure you have installed the correct versions of all of the libraries. To install all the libraries, make sure you ran `pip install -r requirements.txt` in your Conda environment.

Simply run the cell below. You can click on the cell and press `⇧↩` (Shift + Enter) to run it. If you see any warning/error messages, please make sure you have followed the installation instructions in the [README](https://github.gatech.edu/omscs6601/assignment_2/blob/master/README.md). In case you can't resolve them please check out Edstem thread dedicated to "Assignment 2".

In [1]:
%load_ext autoreload
%autoreload 2
%run helpers/verify_config.py # verify the environment setup

Your python version is  3.12.7
✅ ALL GOOD


## About the Game

The rules of Trail Isolation are a variation of the original Isolation game. In the original form of the game there are two players, each with their own game piece, and an (N x N) grid of squares. At the beginning of the game, the first player places their piece on any square. The second player follows suit, and places their piece on any one of the available squares. From that point on, the players alternate turns moving their piece like a **queen in chess** (any number of open squares vertically, horizontally, or diagonally).  

In standard Isolation, after the piece is moved, the square that was previously occupied is blocked permanently, and cannot be used for the remainder of the game. The first player who is unable to move their queen loses.

In this variant called **Trail Isolation**, queens still move like chess queens, but there is an additional twist:  
- The **square the queen moved from** is blocked permanently (same as standard Isolation).  
- The **squares the queen passed through during the move** (the trail) become **temporarily blocked** for the opponent’s next turn only.  
- After the opponent completes their move, those trail squares automatically open back up.  

This means that each move leaves behind a trail that briefly limits the opponent’s options, introducing new strategic considerations.

For clarity, examine the scenario below:

---

Below is a depiction of the empty starting board position.

<div>
<img src="./img/trail_0.png" width="500"/>
</div>

Q1 places their queen on the board, and Q2 follows suit. At this position, Q1 has many potential future moves (shown in light blue).

<div>
<img src="./img/trail_1.png" width="500"/>
</div>

Q1 decides to move three spaces to the right. The square behind (black) becomes permanently blocked, while the intermediate squares (purple) form a **trail** that is blocked for Q2’s next turn only. The orange shows open spots available to Q2 this turn.

<div>
<img src="./img/trail_2.png" width="500"/>
</div>

Q2 then moves diagonally up-left. Their starting square becomes permanently blocked (black), and their path squares form a temporary trail (purple). Meanwhile, Q1’s earlier trail is now cleared and those squares are again open.

<div>
<img src="./img/trail_3.png" width="500"/>
</div>

After several turns, the board fills with permanent blocks. In this example, Q2 has no available moves left, so Q1 wins.

<div>
<img src="./img/trail_4.png" width="500"/>
</div>


### For more clarity, you can try playing the game against the Random Player or against yourself using the interactive tool below

In [10]:
!pip install ipywidgets==8.1.1

In [12]:
!pip install notebook==7.0.7

  Using cached notebook-7.0.7-py3-none-any.whl.metadata (10 kB)
  Using cached fqdn-1.5.1-py3-none-any.whl.metadata (1.4 kB)
  Using cached isoduration-20.11.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached uri_template-1.3.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached webcolors-24.11.1-py3-none-any.whl.metadata (2.2 kB)
Using cached notebook-7.0.7-py3-none-any.whl (4.0 MB)
Using cached webcolors-24.11.1-py3-none-any.whl (14 kB)
Using cached fqdn-1.5.1-py3-none-any.whl (9.1 kB)
Using cached isoduration-20.11.0-py3-none-any.whl (11 kB)
Using cached uri_template-1.3.0-py3-none-any.whl (11 kB)
  Attempting uninstall: notebook
    Found existing installation: notebook 7.2.2
    Uninstalling notebook-7.2.2:
      Successfully uninstalled notebook-7.2.2


In [3]:
# Following two lines make sure anything imported from .py scripts 
# is automatically reloaded if edited & saved (e.g. local unit tests or players)
%load_ext autoreload
%autoreload 2
from board_viz import ReplayGame, InteractiveGame
from isolation import Board
from test_players import RandomPlayer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# replace RandomPlayer() with None if you want to play for both players. Or, try playing against your bot CustomPlayer(), once you have it implemented.
#ig = InteractiveGame(RandomPlayer(), show_legal_moves=True)
ig = InteractiveGame(None, show_legal_moves=True)

### One other thing you can do is simulate a game between two players and replay it.

Run the next cell, click inside the text input box right above the slider and press Up or Down.

In [5]:
# Here is an example of how to visualise a game replay of 2 random players
game = Board(RandomPlayer(), RandomPlayer())
winner, move_history, termination = game.play_isolation(time_limit=1000, print_moves=False)

bg = ReplayGame(game, move_history, show_legal_moves=True)
bg.show_board()

GridspecLayout(children=(GridspecLayout(children=(Button(description=' ', layout=Layout(grid_area='widget001',…

## Assignment contents

In this assignment you will need to implement evaluation functions and game playing methods. Your goal is to implement the following parts of the notebook:

1. Evaluation functions (`OpenMoveEvalFn` and `CustomEvalFn` if you wish to use the latter)
2. The minimax algorithm (`minimax`)
3. Alpha-beta pruning (`alphabeta`)
4. Adjust the `move()` according to section you are trying to work on.

### Evaluation Functions

These functions will inform the value judgements your AI will make when choosing moves. There are 2 classes:

- `OpenMoveEvalFn` - Returns the number of available moves open for your player minus the number of moves available for opponent player. All baseline tests will use this function. **This is mandatory**
- `CustomEvalFn` - You are encouraged to create your own evaluation function here.

#### Notes on these classes
1. You may write additional code within each class. However, we will only be invoking the `score()` function. You may not change the signature of this function.
2. When writing additional code please try not to copy the existing cells since they contain `#export` comments that is used for converting the notebook to `submission.py` file. 

### CustomPlayer

This is the meat of the assignment. A few notes about the class:

- You are permitted to change the default values within the function signatures provided. In fact, when you have your custom evaluation function, you are encouraged to change the default values for `__init__` to use the new eval function.
- You are free change the contents of each of the provided methods. When you are ready with `alphabeta()`, for example, you should update `move()` to use that function instead.
- You are free to add more methods to the class.
- You may not create additional external functions and classes that are referenced from within this class.

Your agent will have a limited amount of time to act each turn (1 second). We will call these functions directly so **don’t modify** the function names and their parameter order.

We have divided the tests into three sections (mentioned in details in next grading section below), each with their own submission limit.

These are the bare minimum requirements for your AI, and the rest is up to you. You will be scored according to how well your AI performs against some baseline AIs that we provide (see [Grading](#Grading-&-submission-policies)). If you want to improve over the base performance, here are a few suggestions:

- Use partition techniques.
- Store the evaluation scores for past moves.
- Modify your evaluation function to account for “killer moves”.
- Optimize functions that are called often.
- Order nodes to maximize pruning.

We will provide you checkpoints and instructions below once you are ready to submit for each of these sections.

# Exporting the notebook

In order to do get your submission file ready you will need to make sure have **saved your notebook** and run:

In [25]:
 %run helpers/notebook2script section2a

C:\Users\Hp\Downloads\attachment_1 (1)\assignment2\helpers\notebook2script.py:15: SyntaxWarning: invalid escape sequence '\s'
  matchList = [re.search("^\s*tests", cell) for cell in cellText] # If any spaces are in front


Late Policy:
    
      "I have read the late policy for CS3600/6601."
    


Please type 'yes' to agree and continue> yes




Honor Pledge:
    
      "I have read the Collaboration and Academic Honesty policy for CS3600/6601.
      I certify that I have or will use outside references only in accordance with
      this policy, that I have or will cite any such references via code comments,
      and that I have not or will not copy any portion of my submission from another
      past or current student."

    


Please type 'yes' to agree and continue> yes




Converted notebook.ipynb to section2a\submission.py


Once execution is complete open autogenerated `submission.py` and verify that it contains all of the imports, functions and classes you are required to implement. Only then you can proceed to the Gradescope for submission.

**Do NOT erase the `#export` at the top of any cells as it is used by `notebook2script.py` to extract cells for submission.**

# Coding time!

## Importing External Modules

In [27]:
# Following two lines make sure anything imported from .py scripts 
# is automatically reloaded if edited & saved (e.g. local unit tests or players)
%load_ext autoreload
%autoreload 2
import player_submission_tests as tests
from test_players import HumanPlayer, RandomPlayer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
#export
import time
from isolation import Board

If you have discussed this assignment at a whiteboard level, got help from an Ed discussion or have used external resources (not provided by the instructors) that you may want to cite, please do so in the cell below as a python comment! (no need to cite python or included packages documentation)

In [ ]:
#export
# Credits if any
# 1)
# 2)
# 3)

## OpenMoveEvalFn
- This is where you write your evaluation function to evaluate the state of the board.
- The test cases below the code are expected to pass locally before you make a submission.
- Hints: Remember when calling the below helpful methods that you do need to inform both methods of who your player is (consult those methods' docstrings for more information).

Here are a couple methods you might find useful to implement `OpenMoveEvalFn()`:

In [31]:
Board.get_player_moves??

Signature: Board.get_player_moves(self, my_player=None)
Source:   
    def get_player_moves(self, my_player=None):
        """
        Get all legal moves of certain player object. Should pass in yourself to get your moves.
        Parameters:
            my_player (Player), Player to get moves for
            If calling from within a player class, my_player = self can be passed.
        returns
            [(int, int)]: List of all legal moves. Each move takes the form of
            (row, column).

        """
        if (my_player == self.__player_1__ and self.__active_player__ == self.__player_1__):
            return self.get_active_moves()
        elif (my_player == self.__player_1__ and self.__active_player__ != self.__player_1__):
            return self.get_inactive_moves()
        elif (my_player == self.__player_2__ and self.__active_player__ == self.__player_2__):
            return self.get_active_moves()
        elif (my_player == self.__player_2__ and self.__active_playe

In [33]:
Board.get_opponent_moves??

Signature: Board.get_opponent_moves(self, my_player=None)
Source:   
    def get_opponent_moves(self, my_player=None):
        """
        Get all legal moves of the opponent of the player provided. Should pass in yourself to get your opponent's moves.
        If calling from within a player class, my_player = self can be passed.
        Parameters:
            my_player (Player), The player facing the opponent in question
            If calling from within a player class, my_player = self can be passed.
        returns
            [(int, int)]: List of all opponent's moves. Each move takes the form of
            (row, column).

        """
        if (my_player == self.__player_1__ and self.__active_player__ == self.__player_1__):
            return self.get_inactive_moves()
        if (my_player == self.__player_1__ and self.__active_player__ != self.__player_1__):
            return self.get_active_moves()
        elif (my_player == self.__player_2__ and self.__active_player__ == s

In [35]:
#export
class OpenMoveEvalFn:
    def score(self, game, my_player=None):
        """Score the current game state
        Evaluation function that outputs a score equal to how many
        moves are open for AI player on the board minus how many moves
        are open for Opponent's player on the board.

        Note:
            If you think of better evaluation function, do it in CustomEvalFn below.

            Args
                game (Board): The board and game state.
                my_player (Player object): This specifies which player you are.

            Returns:
                float: The current state's score. MyMoves-OppMoves.

            """

        # Get number of moves available to the current player
        my_moves = len(game.get_player_moves(my_player))
        
        # Get number of moves available to the opponent
        opp_moves = len(game.get_opponent_moves(my_player))
        
        return my_moves - opp_moves


######################################################################
########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################
######## IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ############
######################################################################
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######
tests.correctOpenEvalFn(OpenMoveEvalFn)
################ END OF LOCAL TEST CODE SECTION ######################


OpenMoveEvalFn Test: This board has a score of 15.



## About the local test above
You should look at the test in `player_submission_tests.py` to verify that your evaluation function is correct. If you want to edit the test (which you most definitely can), then edit the source code back in `player_submission_tests.py`.

---

## CustomPlayer
- CustomPlayer is the player object that will be used to play the game of isolation.
- The `move()` method will be used to pass over to you the current state of the game board.
- The content of the `move()` method will be changed by you according to the section you are attempting to pass. While you can use Iterative Deepening & Alpha-Beta (ID+AB) to beat our agents in all of the sections, going directly for ID+AB is error prone. As such, we highly recommend you to start with MiniMax (MM), then implement Alpha-Beta (AB), and only then go for ID+AB.
- By default, right now `move()` calls `minimax()` as you can see below.
- You are not allowed to modify the function signatures or class signatures we provide. However, in case you want to have an additonal parameter you can do it at the very end of parameter list (see examples below). However, it must have a default value and you shouldn't expect it to be passed on the server-side (i.e. Gradescope). Thus, Gradescope will be using the default value.

Originally:
```python
def move(self, game, time_left):
    ...
```
Adding a new argument with default parameter.
```python
def move(self, game, time_left, new_parameter=default_value):
    ...
```

Don't do this, you will get an error in the auto-grader and lose your submission:
```python
def move(self, game, time_left, new_parameter):
    ...
```
```python
def move(self, new_parameter, game, time_left):
    ...
```


In [37]:
#export
class CustomPlayer:
    # TODO: finish this class!
    """Player that chooses a move using your evaluation function
    and a minimax algorithm with alpha-beta pruning.
    You must finish and test this player to make sure it properly
    uses minimax and alpha-beta to return a good move."""

    def __init__(self, search_depth=2, eval_fn=OpenMoveEvalFn()):
        """Initializes your player.
        
        if you find yourself with a superior eval function, update the default
        value of `eval_fn` to `CustomEvalFn()`
        
        Args:
            search_depth (int): The depth to which your agent will search
            eval_fn (function): Evaluation function used by your agent
        """
        self.eval_fn = eval_fn
        self.search_depth = search_depth
    
    def move(self, game, time_left):
        """Called to determine one move by your agent

        Note:
            1. Do NOT change the name of this 'move' function. We are going to call
            this function directly.
            2. Call alphabeta instead of minimax once implemented.
        Args:
            game (Board): The board and game state.
            time_left (function): Used to determine time left before timeout

        Returns:
            tuple: (int,int): Your best move
        """
        best_move, utility = minimax(self, game, time_left, depth=self.search_depth)
        return best_move

    

    def utility(self, game, my_turn):
        """You can handle special cases here (e.g. endgame)"""
        
        return self.eval_fn.score(game, self)



###################################################################
########## DON'T WRITE ANY CODE OUTSIDE THE CLASS! ################
###### IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ###########
###################################################################

## Minimax
- This is where you will implement the minimax algorithm. The final output of your minimax should come from this method and this is the only method that Gradescope will call when testing minimax.
- With MM implemented you are expected to pass: **Defeat a Random Player >=90% of the time.**
- Useful functions: The useful methods will probably all come from isolation.py. A couple of particularly interesting ones could be `forecast_move()` and your `score()` method from OpenMoveEvalFn. Remember the double question mark trick from Assignment 0 if you feel you are flipping between files too much!
 
> **Recommended (Approved) Resources**
> - Canvas Game Playing Modules
> - Course AIMA Textbook 4th Edition, Section 5.1 through 5.2

In [39]:
#export
def minimax(player, game, time_left, depth, my_turn=True):
    """Implementation of the minimax algorithm.
    Args:
        player (CustomPlayer): This is the instantiation of CustomPlayer()
            that represents your agent. It is used to call anything you
            need from the CustomPlayer class (the utility() method, for example,
            or any class variables that belong to CustomPlayer()).
        game (Board): A board and game state.
        time_left (function): Used to determine time left before timeout
        depth: Used to track how deep you are in the search tree
        my_turn (bool): True if you are computing scores during your turn.

    Returns:
        (tuple, int): best_move, val
    """

    # Base case: if we've reached the depth limit or game is over
    if depth == 0 or not game.get_active_moves():
        return None, player.utility(game, my_turn)
    
    best_move = None
    
    if my_turn:
        # Maximizing player
        best_value = float('-inf')
        legal_moves = game.get_active_moves()
        
        for move in legal_moves:
            # Forecast the move to get the new game state
            new_game, is_over, winner = game.forecast_move(move)
            
            # Recursive call for the opponent's turn
            _, value = minimax(player, new_game, time_left, depth-1, False)
            
            # Update best value and move
            if value > best_value:
                best_value = value
                best_move = move
                
        return best_move, best_value
        
    else:
        # Minimizing player
        best_value = float('inf')
        legal_moves = game.get_active_moves()
        
        for move in legal_moves:
            # Forecast the move to get the new game state
            new_game, is_over, winner = game.forecast_move(move)
            
            # Recursive call for our turn
            _, value = minimax(player, new_game, time_left, depth-1, True)
            
            # Update best value and move
            if value < best_value:
                best_value = value
                best_move = move
                
        return best_move, best_value

######################################################################
########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################
######## IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ############
######################################################################
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######
tests.beatRandom(CustomPlayer)
tests.minimaxTest(CustomPlayer, minimax)  
################ END OF LOCAL TEST CODE SECTION ######################
   



 CustomPlayer - Q1  Turn
move chosen:  (4, 4)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |  |  |  |  |  |  |  |  |
1 |  |Q2|  |  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |  |Q1|  |  |  |  |
5 |  |  |  |  |><|  |  |  |  |
6 |  |  |  |  |  |  |  |  |  |
7 |  |  |  |  |  |  |  |  |  |
8 |  |  |  |  |  |  |  |  |  |

 RandomPlayer - Q2  Turn
move chosen:  (2, 0)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |  |  |  |  |  |  |  |  |
1 |  |><|  |  |  |  |  |  |  |
2 |Q2|  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |  |
4 |  |  |  |  |Q1|  |  |  |  |
5 |  |  |  |  |><|  |  |  |  |
6 |  |  |  |  |  |  |  |  |  |
7 |  |  |  |  |  |  |  |  |  |
8 |  |  |  |  |  |  |  |  |  |

 CustomPlayer - Q1  Turn
move chosen:  (2, 4)
  |0 |1 |2 |3 |4 |5 |6 |7 |8 |
0 |  |  |  |  |  |  |  |  |  |
1 |  |><|  |  |  |  |  |  |  |
2 |Q2|  |  |  |Q1|  |  |  |  |
3 |  |  |  |  |- |  |  |  |  |
4 |  |  |  |  |><|  |  |  |  |
5 |  |  |  |  |><|  |  |  |  |
6 |  |  |  |  |  |  |

---
## Section 2a Checkpoint
### Now it's a good time to submit for Section2a - See [Exporting the notebook](#Exporting-the-notebook)

In case you want to submit please uncomment and run the cell below.

Your code will be generated in the folder named `section2a`, please upload `submission.py` file to [Gradescope](https://www.gradescope.com/)

In [41]:
 %run helpers/notebook2script section2a

C:\Users\Hp\Downloads\attachment_1 (1)\assignment2\helpers\notebook2script.py:15: SyntaxWarning: invalid escape sequence '\s'
  matchList = [re.search("^\s*tests", cell) for cell in cellText] # If any spaces are in front


Late Policy:
    
      "I have read the late policy for CS3600/6601."
    


Please type 'yes' to agree and continue> yes




Honor Pledge:
    
      "I have read the Collaboration and Academic Honesty policy for CS3600/6601.
      I certify that I have or will use outside references only in accordance with
      this policy, that I have or will cite any such references via code comments,
      and that I have not or will not copy any portion of my submission from another
      past or current student."

    


Please type 'yes' to agree and continue> yes




Converted notebook.ipynb to section2a\submission.py


---

## AlphaBeta
- This is where you will implement the alphabeta algorithm. The final output of your alphabeta should come from this method.
- With A/B implemented you are expected to pass: **Minimax level 2 >= 70% of the time**
- Useful functions: The useful methods will probably all come from `isolation.py`. A couple of particularly interesting ones could be `forecast_move()` and your `score()` method from OpenMoveEvalFn. Remember the double question mark trick from Assignment 0 if you feel you are flipping between files too much!
 
> **Recommended (Approved) Resources**
> - Canvas Game Playing Modules
> - Course AIMA Textbook 4th Edition, Section 5.3

In [43]:
#export
def alphabeta(player, game, time_left, depth, alpha=float("-inf"), beta=float("inf"), my_turn=True):
    """Implementation of the alphabeta algorithm.
    
    Args:
        player (CustomPlayer): This is the instantiation of CustomPlayer() 
            that represents your agent. It is used to call anything you need 
            from the CustomPlayer class (the utility() method, for example, 
            or any class variables that belong to CustomPlayer())
        game (Board): A board and game state.
        time_left (function): Used to determine time left before timeout
        depth: Used to track how deep you are in the search tree
        alpha (float): Alpha value for pruning
        beta (float): Beta value for pruning
        my_turn (bool): True if you are computing scores during your turn.
        
    Returns:
        (tuple, int): best_move, val
    """
    
    # Base case: if we've reached the depth limit or game is over
    if depth == 0 or not game.get_active_moves():
        return None, player.utility(game, my_turn)
    
    best_move = None
    
    if my_turn:
        # Maximizing player
        best_value = float('-inf')
        legal_moves = game.get_active_moves()
        
        for move in legal_moves:
            # Forecast the move to get the new game state
            new_game, is_over, winner = game.forecast_move(move)
            
            # Recursive call for the opponent's turn
            _, value = alphabeta(player, new_game, time_left, depth-1, alpha, beta, False)
            
            # Update best value and move
            if value > best_value:
                best_value = value
                best_move = move
                
            # Alpha-beta pruning
            alpha = max(alpha, best_value)
            if beta <= alpha:
                break  # Beta cutoff
                
        return best_move, best_value
        
    else:
        # Minimizing player
        best_value = float('inf')
        legal_moves = game.get_active_moves()
        
        for move in legal_moves:
            # Forecast the move to get the new game state
            new_game, is_over, winner = game.forecast_move(move)
            
            # Recursive call for our turn
            _, value = alphabeta(player, new_game, time_left, depth-1, alpha, beta, True)
            
            # Update best value and move
            if value < best_value:
                best_value = value
                best_move = move
                
            # Alpha-beta pruning
            beta = min(beta, best_value)
            if beta <= alpha:
                break  # Alpha cutoff
                
        return best_move, best_value


######################################################################
########## DON'T WRITE ANY CODE OUTSIDE THE FUNCTION! ################
######## IF YOU WANT TO CALL OR TEST IT CREATE A NEW CELL ############
######################################################################
##### CODE BELOW IS USED FOR RUNNING LOCAL TEST DON'T MODIFY IT ######
# tests.name_of_the_test #you can uncomment this line to run your test
################ END OF LOCAL TEST CODE SECTION ######################

## About the lack of a local test above
Notice that we do not have any code here. We want you to learn to write your own test cases, so feel free to get creative! You can always create the test in `player_submission_tests.py` and then run it over here in a manner identical to how local tests have been run so far.

**IMPORTANT**

Now remember that the server (i.e. Gradescope) uses `move()` to interface with your code. So now you will need to update the `move()` method (which you saw earlier in the CustomPlayer class) to call `alphabeta()` so as to return the best move.

---
## Section 2b Checkpoint
### Now it's a good time to submit for Section2b - See [Exporting the notebook](#Exporting-the-notebook)

In case you want to submit please uncomment and run the cell below.

Your code will be generated in the folder named `section2b`. Please upload `submission.py` file to [Gradescope](https://www.gradescope.com/)

In [45]:
 %run helpers/notebook2script section2b

C:\Users\Hp\Downloads\attachment_1 (1)\assignment2\helpers\notebook2script.py:15: SyntaxWarning: invalid escape sequence '\s'
  matchList = [re.search("^\s*tests", cell) for cell in cellText] # If any spaces are in front


Late Policy:
    
      "I have read the late policy for CS3600/6601."
    


Please type 'yes' to agree and continue> yes




Honor Pledge:
    
      "I have read the Collaboration and Academic Honesty policy for CS3600/6601.
      I certify that I have or will use outside references only in accordance with
      this policy, that I have or will cite any such references via code comments,
      and that I have not or will not copy any portion of my submission from another
      past or current student."

    


Please type 'yes' to agree and continue> yes




Converted notebook.ipynb to section2b\submission.py


---

## That does not cover all 100 points though!
- You're right, and that's on purpose. Each of the bullets below try to walk you through how you may want to think about beating the remaining agents.
    - First up is the alphabeta agent. Vanilla alphabeta (that is, alphabeta with no optimization) may not do so well against this agent. However, any agent that searches deeper with the same algorithm probably has a bigger advantage. You may learn about a method that allows your algorithm to search in such a way that you can find the maximum search depth without running out of time. This will probably come up in class or you can read through the book to find out what you are looking for.
    - Next to beat is the agent with iterative deepening. This one is a little harder to think about, given that you may have used all the tools that you may think of to try a make a "better" agent. But you may have just implemented the evaluation function that was discussed in class. Maybe we can do better - like checking for winning moves and prioritizing those! Or if you are feeling really creative, you can always try editing the `CustomEvalFn` below this cell and come up with an awesome idea of your own.
    - Now to Peter's agent with the secret evaluation function. Here we have nothing to tell you. Use everything in your toolbox and within the class rules to defeat it. This is by far the hardest 5 points to get! Good luck and have fun!
    
- Remember that you may want to edit the methods in the cell with the `CustomPlayer` class to try and implement some of the above. You are certainly free to as long as you adhere to the general rules about editing provided code (which can be found by reading the cell above the `CustomPlayer` code).

## CustomEvalFn
- Edit the below to come up with your very own improved evaluation function. The typical rules about how you can and cannot edit the code we have given (namely, the function signature rules) apply here.
- **IMPORTANT**: There's one big thing to keep in mind when the below is exported to `submission.py`. When the export happens, your resulting `submission.py` is parsed top-down, so you may have errors when trying to run that file with a custom evaluation function.
    - The fix is to make sure this does not happen is to follow these steps: Use "Edit->Move Cell Up" to move the below cell to just above the first time you call CustomEvalFn (probably in CustomPlayer) -> Now run `helpers/notebook.ipynb` -> Submit the resulting `submission.py` to Gradescope to test your submission.

In [47]:
#export
class CustomPlayer:
    def __init__(self, search_depth=3, eval_fn=OpenMoveEvalFn()):
        self.eval_fn = eval_fn
        self.search_depth = search_depth
    
    def move(self, game, time_left):
        """Called to determine one move by your agent"""
        # Use alpha-beta pruning instead of minimax
        best_move, utility = alphabeta(self, game, time_left, depth=self.search_depth)
        return best_move

    def utility(self, game, my_turn):
        """You can handle special cases here (e.g. endgame)"""
        if not game.get_active_moves():
            if my_turn:
                return float('-inf')
            else:
                return float('inf')
        return self.eval_fn.score(game, self)

Now you may need to change the `move()` method again in the CustomPlayer class. In addition, you may also need to edit `eval_fn()` in CustomPlayer to have your agent use the above custom evaluation function when it is playing against the test agents.

---
## Section 2c Checkpoint
### Now it's a good time to submit for Section2c - See [Exporting the notebook](#Exporting-the-notebook)

In case you want to submit please uncomment and run the cell below.

Your code will be generated in the folder named `section2c`. Please upload `submission.py` file to [Gradescope](https://www.gradescope.com/)

In [49]:
 %run helpers/notebook2script section2c

C:\Users\Hp\Downloads\attachment_1 (1)\assignment2\helpers\notebook2script.py:15: SyntaxWarning: invalid escape sequence '\s'
  matchList = [re.search("^\s*tests", cell) for cell in cellText] # If any spaces are in front


Late Policy:
    
      "I have read the late policy for CS3600/6601."
    


Please type 'yes' to agree and continue> yes




Honor Pledge:
    
      "I have read the Collaboration and Academic Honesty policy for CS3600/6601.
      I certify that I have or will use outside references only in accordance with
      this policy, that I have or will cite any such references via code comments,
      and that I have not or will not copy any portion of my submission from another
      past or current student."

    


Please type 'yes' to agree and continue> yes




Converted notebook.ipynb to section2c\submission.py


---